In [1]:
from __future__ import division
from __future__ import print_function
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
import networkx as nx
from collections import defaultdict
import pandas as pd
import numpy  as np
import random

In [2]:
# BELOW CODE WAS IMPLEMENTED BY John Reid in pybool package
# https://github.com/JohnReid/pybool/blob/master/python/pybool/chow_liu_trees.py
# The code has been pasted here because csgrads1 was having issues installing pybool
# Add one laplace smoothing has also been included.

def marginal_distribution(X, u):
    """
    Return the marginal distribution for the u'th features of the data points, X.
    """
    values = defaultdict(float)
    s = 1. / len(X)
    for x in X:
        values[x[u]] += s
    return values



def marginal_pair_distribution(X, u, v):
    """
    Return the marginal distribution for the u'th and v'th features of the data points, X.
    """
    if u > v:
        u, v = v, u
    values = defaultdict(float)
    s = 1. / len(X)
    for x in X:
        values[(x[u], x[v])] += s
    return values



def calculate_mutual_information(X, u, v):
    """
    X are the data points.
    u and v are the indices of the features to calculate the mutual information for.
    """
    if u > v:
        u, v = v, u
    marginal_u = marginal_distribution(X, u)
    marginal_v = marginal_distribution(X, v)
    marginal_uv = marginal_pair_distribution(X, u, v)
    I = 0.
    for x_u, p_x_u in marginal_u.iteritems():
        for x_v, p_x_v in marginal_v.iteritems():
            if (x_u, x_v) in marginal_uv:
                p_x_uv = marginal_uv[(x_u, x_v)]
                I += p_x_uv * (np.log2(p_x_uv) - np.log2(p_x_u) - np.log2(p_x_v))
    return I


def build_chow_liu_tree(X, n):
    """
    Build a Chow-Liu tree from the data, X. n is the number of features. The weight on each edge is
    the negative of the mutual information between those features. The tree is returned as a networkx
    object.
    """
    G = nx.Graph()
    for v in xrange(n):
        G.add_node(v)
        for u in xrange(v):
            G.add_edge(u, v, weight=-calculate_mutual_information(X, u, v))
    T = nx.minimum_spanning_tree(G)
    return T

In [3]:
data_dir = "./small-10-datasets/"
data_titles = ['accidents', 'baudio', 'bnetflix', 'dna', 'jester', 'kdd', 'msnbc',
              'nltcs', 'plants', 'r52']
test  = dict()
train = dict()
valid = dict()

for title in data_titles:
    test[title]  = np.loadtxt(data_dir + title + '.test.data', delimiter=',')
    train[title] = np.loadtxt(data_dir + title + '.ts.data', delimiter=',')
    valid[title] = np.loadtxt(data_dir + title + '.valid.data', delimiter=',')

In [60]:
# Log Likelihood
def LL(p):
    return np.sum(np.log2(p))

def AVG_LL(P):
    return sum([LL(p) for p in P]) / len(P)

# Log Sum Exponent
def lse(a):
    m = max(a)
    return np.log2(np.sum(np.power(2., a - m))) + m

def Split_Tree(T, k):
    split = [i for i in range(0, len(T), int(len(T) / k))]
    return [T[range(split[i],len(T) if i == k -1 else split[i+1])] for i in range(k)]

# Transforms MST into a DAG, and then finds associated probabilities as well.
def Create_Network(MST, T):
    n = len(MST)
    T = np.transpose(T)
    network = [{} for i in range(n)]
    root = 0

    p = (sum(T[root] == 1) + 1) / (len(T[root]) + 2)
    network[root] = {root : [1 - p, p]}
    children = np.ndarray.flatten(np.argwhere(np.transpose(MST[root]) != 0))
    parents = {root : children}
    for c in children:
        MST[c][root] = 0.

    while (len(parents) != 0):
        newParents   = {}
        for parent in parents:
            children = parents[parent]
            for c in children:
                # Remove edge, make directed
                p = [(sum(T[c][T[parent] == 0] == 1) + 1) / (sum(T[parent] == 0) + 2),
                     (sum(T[c][T[parent] == 1] == 1) + 1) / (sum(T[parent] == 1) + 2)]
                network[c].update({parent : [1 - p[0], p[0], 1 - p[1], p[1]]})
                cc = np.ndarray.flatten(np.argwhere(np.transpose(MST[c]) != 0))
                for child in cc:
                    MST[child][c] = 0
                if (len(children) != 0):
                    newParents.update({c : cc})
        parents = newParents    
    return network;

# Predicts a network generated from above.
def Predict_Network(N, test):
    all_predictions = []
    for t in test:
        predictions = []
        for i in range(len(N)):
            probs = []
            for k in N[i].keys():
                if(k == i):
                    probs.append(N[i][k][0] if t[i] == 0 else N[i][k][1])
                else:
                    if (t[k] == 0):
                        probs.append(N[i][k][0] if t[i] == 0 else N[i][k][1])
                    else:
                        probs.append(N[i][k][2] if t[i] == 0 else N[i][k][3])
            predictions.append(np.product(probs))
        all_predictions.append(predictions)
    return np.array(all_predictions)

def Predict_Mixture(M, test):
    N  = M[1]
    pi = M[0]
    n = len(pi)
    predictions = []
    for i in range(n):
        predictions.append(pi[i] * Predict_Network(N[i], test))
    return np.sum(predictions, axis=0)
    
# Bayesian Network, No edges Algorithm
# Takes in a dataset of binary variables and takes a test set of binary variables.
def BN_NE(T, test):
    cols = np.transpose(T)
    n    = len(cols)
    p_1 = np.array([(sum(cols[i] == 1) + 1) / (len(cols[i]) + 2) for i in range(n)])
    return np.array([[p_1[+i] if ti == 0 else 1 - p_1[i] for i, ti in enumerate(t)] for t in test])

# Bayesian Network, Chow-Liu Algorithm
# Returns a data structure in the form of a Bayesian Network with probabilities pre-computed inside.
def BN_CL(T):
    n = len(T[0])
    CLT = Create_Network(np.array(nx.adjacency_matrix(build_chow_liu_tree(T, n)).todense()), T)
    return CLT

# Mixtures of Tree Bayesian networks using EM
def MT_BN(T, k):
    V = T[range(0, int(len(T) / 10))]
    T     = T[range(int(len(T) / 10), len(T))]
    return EM(T, k, 10)

# Expectation Maximization
def EM(T, k, max_iter):
    split = Split_Tree(T, k)
    r = np.array([random.random() for i in range(k)]).astype(np.float64)
    pi = [i / sum(r) for i in r]
    N = [BN_CL(s) for s in split]
    prevPi = []
    it = 0
    while (prevPi != pi and it != max_iter):
        # E-step
        prevPi = pi
        it += 1
        
        W = []
        for ki in range(k):
            p = np.array([AVG_LL(Predict_Network(n, [t])) for n in N]) + np.log2(pi)
            p = np.power(2, p - lse(p))
            W.append(p)

        # M-step
        pk = [np.sum(w) / sum(w) for w in np.transpose(W)]
        return pk

        # Assign points to their most likely cluster, generate a tree from that
        for ki in range(k):
            print(np.ndarray.flatten(np.argwhere(np.array([np.argmax(p) for p in E]) == ki)))
            Ti = T[np.ndarray.flatten(np.argwhere(np.array([np.argmax(p) for p in E]) == ki))]
            if (len(Ti) == 0):
                continue
            N[ki] = BN_CL(Ti)
    return [pi, N]
    

In [213]:
AVG_LL(BN_NE(train['accidents'], test['accidents']))

-303.09757458863464

In [53]:
AVG_LL(Predict_Network((BN_CL(T=train['accidents'])), test['accidents']))

-47.88027600729854

In [ ]:
AVG_LL(Predict_Mixture(MT_BN(train['accidents'], 5), test['accidents']))

In [63]:
MT_BN(train['accidents'][0:20],2)

[1.0000000000000004, 1.0]